In [ ]:
import pandas as pd
import numpy as np
import csv
import random
import matplotlib.pyplot as plt
import seaborn as sns

# import from scripts
import os
#os.chdir(os.path.expanduser("D:\\intr_timescales\\isttc\\scripts"))
os.chdir(os.path.expanduser("C:\\Users\\ipoch\\Documents\\repos\\isttc\\scripts"))
from calculate_acf import acf_pearsonr_trial_avg, acf_sttc_trial_avg

### Get and prep the data

In [ ]:
area = 'pfdl' # pfp

In [ ]:
# binned data
results_folder = 'D:\\projects_q_30_10_2024\\isttc\\results\\monkey\\'
save_folder_binned = results_folder + 'fixation_period_1000ms\\binned\\' + area + '\\acf\\'

csv_data_file = results_folder + 'data_' + area + '_fixon_1500ms_fixation_with_empty_binned_50ms.csv'
with open(csv_data_file, newline='') as f:
    reader = csv.reader(f)
    sua_binned_list = list(reader)
    
n_binned_spike_trains = len(sua_binned_list)
print('N spike_trains in {}: {}'.format(area, n_binned_spike_trains))

# transform list to an array and to a dataframe 
sua_binned_array = np.array(sua_binned_list)
sua_binned_array = sua_binned_array[:, :-1-9] # for 1000 calc

bin_cols = ['bin_' + str(i) for i in range(sua_binned_array.shape[1]-3)]
sua_binned_df = pd.DataFrame(sua_binned_array, columns=['unit_id', 'trial_id','condition_id'] + bin_cols)
sua_binned_df = sua_binned_df.astype('int')

n_binned_units = len(sua_binned_df['unit_id'].unique())
print('n units {}'.format(n_binned_units))

sua_binned_df.head(2)

In [ ]:
# non-binned data
results_folder = 'D:\\projects_q_30_10_2024\\isttc\\results\\monkey\\'
save_folder_non_binned = results_folder + 'fixation_period_1000ms\\non_binned\\' + area + '\\acf\\'

csv_data_file = results_folder + 'data_' + area + '_fixon_1500ms_with_empty_fixation.csv'
with open(csv_data_file, newline='') as f:
    reader = csv.reader(f)
    sua_non_binned_list = list(reader)
    
n_non_binned_spike_trains = len(sua_non_binned_list)
print('N spike_trains in {}: {}'.format(area, n_non_binned_spike_trains))

# transform data to a dict, key is unit_id, values is a list of spike trains (one spike train per trial)
units_dict = {}
for spike_train in sua_non_binned_list:
    spike_train_ = np.asarray(spike_train[3:]).astype(int)
    spike_train_1000 = spike_train_[spike_train_ <= 1000]
    if int(spike_train[0]) in units_dict:
        units_dict[int(spike_train[0])].append(spike_train_1000)
    else:
        units_dict[int(spike_train[0])] = []
        units_dict[int(spike_train[0])].append(spike_train_1000)

n_non_binned_units = len(units_dict)
print('n units {}'.format(n_non_binned_units))

### Calculate autocorrelation function

In [ ]:
n_lags = 20
acf_cols = ['acf_' + str(i) for i in range(n_lags)]
print('acf_cols {}'.format(acf_cols))

#### Using Pearson (as in papers)

In [ ]:
n_runs = 50

for i in range(n_runs):
    print('######### RUNNING {}'.format(i))
    acf_average_trial_pearsonr_l = []
    acf_matrix_pearsonr_l = []
    
    unit_id_l = sua_binned_df['unit_id'].unique()
    unit_id_calc_l = []
    
    for unit in unit_id_l:
        print('Processing unit {}'.format(unit))
        sua_binned_unit_df = sua_binned_df.query('unit_id == @unit')
        print('N trials {}'.format(len(sua_binned_unit_df)))
    
        sua_binned_unit_df = sua_binned_unit_df.sample(n=10, replace=True) # for cases when less then 20 trials
        print('N trials {}'.format(len(sua_binned_unit_df)))
        
        if len(sua_binned_unit_df) <= 1:
            print('ONLY 1 TRIAL: can not calculate, skipping...')
        else:
            acf_matrix, acf_average = acf_pearsonr_trial_avg(sua_binned_unit_df[bin_cols].values, n_lags, verbose_=False)
            acf_average_trial_pearsonr_l.append(acf_average)
            acf_matrix_pearsonr_l.append(acf_matrix)
            unit_id_calc_l.append(unit)
    
    acf_average_trial_pearsonr_df = pd.DataFrame(np.array(acf_average_trial_pearsonr_l), columns=acf_cols)
    acf_average_trial_pearsonr_df.insert(0, 'unit_id', unit_id_calc_l)
    
    print('NaNs in acf {}'.format(acf_average_trial_pearsonr_df.isnull().any().any()))
    acf_average_trial_pearsonr_df.head(3)

    acf_average_trial_pearsonr_df.to_pickle(save_folder_binned + 'trials10\\acf_average_trial_pearsonr_with_empty_50ms_20lags_df' + str(i) + '.pkl')
    np.save(save_folder_binned + 'trials10\\acf_matrix_pearsonr_l_with_empty_50ms_20lags_df' + str(i) + '.npy', acf_matrix_pearsonr_l)

    i=i+1

#### Using STTC

In [ ]:
n_runs = 50

for i in range(n_runs):
    print('######### RUNNING {}'.format(i))

    acf_average_trial_sttc_l = []
    acf_matrix_sttc_l = []
    unit_id_calc_l = []
    
    for k,v in units_dict.items():
        print('Processing unit {}, n trials {}'.format(k, len(v)))

        v_subsample = random.choices(v, k=10)
        print('Processing unit {}, n trials {}'.format(k, len(v_subsample)))
        
        if len(v_subsample) <= 1:
            print('ONLY 1 TRIAL: can not calculate, skipping...')
        else:
            acf_matrix, acf_average = acf_sttc_trial_avg(v_subsample, zero_padding_len_=51)
            acf_average_trial_sttc_l.append(acf_average)
            acf_matrix_sttc_l.append(acf_matrix)
            unit_id_calc_l.append(k)
    
    acf_average_trial_sttc_df = pd.DataFrame(np.array(acf_average_trial_sttc_l), columns=acf_cols)
    acf_average_trial_sttc_df.insert(0, 'unit_id', unit_id_calc_l)
    
    print('NaNs in acf {}'.format(acf_average_trial_sttc_df.isnull().any().any()))
    acf_average_trial_sttc_df.head(3)

    acf_average_trial_sttc_df.to_pickle(save_folder_non_binned + 'trials10\\acf_average_trial_sttc_with_empty_50ms_20lags_51padding_df' + str(i) + '.pkl')
    np.save(save_folder_non_binned + 'trials10\\acf_matrix_sttc_l_with_empty_50ms_20lags_51padding_df' + str(i) + '.npy', acf_matrix_sttc_l)

    i=i+1